**<h1>WLASL Dataset Sign Language Model**

**<h3> This notebook aims to use MediaPipe landmarks detection as the starting point** 
    
**<h3> for building an ASL Translation Model**
    
**<h3> Sections:**
* [Importing Libraries](#Importing)
* [Data Preparation](#Preparation)
* [MediaPipe Implementation](#MediaPipe)
* [Visualizing landmarks](#Visualizing)
* [Data Encoding](#Encoding)
* [Data Loading](#Loading)
* [Data Augmentation](#Augmentation)
* [Data Preprocessing](#Preprocessing)
* [Models](#Models)

---

<div id="Importing">
    
**<h2> Importing Libraries** 

In [ ]:
!pip install -q mediapipe==0.10.7

In [ ]:
import os
import gc
import cv2
import json
import time
import spacy
import shutil
import threading
import numpy as np
from tqdm import tqdm
import mediapipe as mp
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from IPython.display import clear_output, FileLink

In [ ]:
# !pip install -q gdown

# # ALL Encoded Lamdmarks V1
# # !gdown 10NXf2LPIBMlI-zrAPIJU7OIsjeXQA3UG

# # ALL Encoded Lamdmarks V2
# !gdown 1tsvPlCJW7sCafqCkESFkgwHl59DQaRrc

# # WLASL Parsed JSON (labels)
# !gdown 1mETzoM47Fo7ggpG-wEYYXOs_rahHfSIK

---

<div id="Preparation">
    
**<h2>Data Preparation   (Done Once)**

<h3> We first load the data

In [ ]:
with open('/kaggle/input/wlasl-processed/WLASL_v0.3.json', 'r') as json_file:
    all_data = json.load(json_file)

<h3> Then we extracting the needed info from the data
<h4>    
    
* gloss : the word being expressed
    
* video_path : the path to the video in the datasets
    
* start_frame : the frame number where the word starts
    
* end_frame : the frame number where each word ends
   
* split : the type of data when modeling (train, val, test)

In [ ]:
video_dir = '/kaggle/input/wlasl-processed/videos'
backup_dir = '/kaggle/input/wlasl2000-resized/wlasl-complete/videos'
data = [] # formatted data

for i in tqdm(range(len(all_data)), ncols=100):
    gloss = all_data[i]['gloss']
    instances = all_data[i]['instances']
    for instance in instances:
        video_id = instance['video_id']
        if os.path.exists(os.path.join(video_dir, f'{video_id}.mp4')):
            video_path = os.path.join(video_dir, f'{video_id}.mp4')
        elif os.path.exists(os.path.join(backup_dir, f'{video_id}.mp4')):
            video_path = os.path.join(backup_dir, f'{video_id}.mp4')
        else:
            continue
            
        frame_start = instance['frame_start']
        frame_end = instance['frame_end']
        split = instance['split']
        data.append({
            'gloss': gloss,
            'video_path': video_path,
            'frame_start': frame_start,
            'frame_end': frame_end,
            'split': split
        })

In [ ]:
len(data)

<h3> We then save the organized dictionary for future uses

In [ ]:
with open('/kaggle/working/WLASL_parsed_data.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)
    
FileLink(r'WLASL_parsed_data.json')

---

<div id="MediaPipe">
    
**<h2>MediaPipe Implementation (Not important when training)**

<h3> We first choose the landmarks
<h4>    
    
* Hands : we'll keep all **42** of them, as they are the most important part.
    
* Pose : **6** landmarks for the upper body excluding the face, as we have dedicated process for it.
    
* Face : out of the **478** landmarks, we'll choose **132**, focusing on the lips, eyes, eyebrows, and the outline of the face.
    
* This brings the total number of landmarks to **180**, each with coordinates (x, y, z).   

In [ ]:
filtered_face = [0, 4, 7, 8, 10, 13, 14, 17, 21, 33, 37, 39, 40, 46, 52, 53, 54, 55, 58,
                 61, 63, 65, 66, 67, 70, 78, 80, 81, 82, 84, 87, 88, 91, 93, 95, 103, 105,
                 107, 109, 127, 132, 133, 136, 144, 145, 146, 148, 149, 150, 152, 153, 154,
                 155, 157, 158, 159, 160, 161, 162, 163, 172, 173, 176, 178, 181, 185, 191,
                 234, 246, 249, 251, 263, 267, 269, 270, 276, 282, 283, 284, 285, 288, 291,
                 293, 295, 296, 297, 300, 308, 310, 311, 312, 314, 317, 318, 321, 323, 324,
                 332, 334, 336, 338, 356, 361, 362, 365, 373, 374, 375, 377, 378, 379, 380,
                 381, 382, 384, 385, 386, 387, 388, 389, 390, 397, 398, 400, 402, 405, 409,
                 415, 454, 466, 468, 473]

filtered_pose = [11, 12, 13, 14, 15, 16]

FACE_NUM = len(filtered_face)
POSE_NUM = len(filtered_pose)
HAND_NUM = 21 # per hand

<h3>Extracting landmarks from a frame

In [ ]:
hands = mp.solutions.hands.Hands()
pose = mp.solutions.pose.Pose()
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

def get_frame_landmarks(frame):
    
    all_landmarks = np.zeros((HAND_NUM * 2 + POSE_NUM + FACE_NUM, 3))
    
    def get_hands(frame):
        results_hands = hands.process(frame)
        if results_hands.multi_hand_landmarks:
            for i, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
                if results_hands.multi_handedness[i].classification[0].index == 0: 
                    all_landmarks[:HAND_NUM, :] = np.array(
                        [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark])
                else:
                    all_landmarks[HAND_NUM:HAND_NUM * 2, :] = np.array(
                        [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark])

    def get_pose(frame):
        results_pose = pose.process(frame)
        if results_pose.pose_landmarks:
            all_landmarks[HAND_NUM * 2:HAND_NUM * 2 + POSE_NUM, :] = np.array(
                [(lm.x, lm.y, lm.z) for lm in results_pose.pose_landmarks.landmark])[filtered_pose]
        
    def get_face(frame):
        results_face = face_mesh.process(frame)
        if results_face.multi_face_landmarks:
            all_landmarks[HAND_NUM * 2 + POSE_NUM:, :] = np.array(
                [(lm.x, lm.y, lm.z) for lm in results_face.multi_face_landmarks[0].landmark])[filtered_face]
        
    threads = []
    threads.append(threading.Thread(target=get_hands, args=(frame,)))
    threads.append(threading.Thread(target=get_pose, args=(frame,)))
    threads.append(threading.Thread(target=get_face, args=(frame,)))
    
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

    return all_landmarks

<h3>Extracting landmarks from a video

In [ ]:
def get_video_landmarks(video_path, start_frame=1, end_frame=-1):
    cap = cv2.VideoCapture(video_path)
    
    # if the starting is 0
    if start_frame <= 1:
        start_frame = 1
        
    # if the video is precropped
    elif start_frame > int(cap.get(cv2.CAP_PROP_FRAME_COUNT)):
        start_frame = 1
        end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
    # if the final frame was not given (-1)    
    if end_frame < 0: 
        end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    num_landmarks = HAND_NUM * 2 + POSE_NUM + FACE_NUM
    all_frame_landmarks = np.zeros((end_frame - start_frame + 1, num_landmarks, 3))
    frame_index = 1
    
    while cap.isOpened() and frame_index <= end_frame:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_index >= start_frame:
            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_landmarks = get_frame_landmarks(frame)
            all_frame_landmarks[frame_index - start_frame] = frame_landmarks

        frame_index += 1

    cap.release()
    hands.reset()
    pose.reset()
    face_mesh.reset()
    return all_frame_landmarks

In [ ]:
def draw_landmarks(input_path, output_path, video_landmarks, start_frame=1, end_frame=-1):
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    if start_frame <= 1:
        start_frame = 1
    elif start_frame > int(cap.get(cv2.CAP_PROP_FRAME_COUNT)):
        start_frame = 1
        end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if end_frame < 0:
        end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
    frame_index = 1
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_index >= start_frame and frame_index <= end_frame:
            frame_landmarks = video_landmarks[frame_index - start_frame]
            landmarks = [(int(x * width), int(y * height)) for x, y, _ in frame_landmarks]
            for x, y in landmarks:
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)
            out.write(frame)
        else:
            # out.write(frame) # Enable if you want the full video
            pass
        frame_index += 1

    cap.release()
    out.release()

---

<div id="Visualizing">
    
**<h2>Visualizing landmarks  (Done Once)**

<h3>Test on a frame

In [ ]:
import requests
from PIL import Image
from io import BytesIO

image_url = 'https://images.unsplash.com/photo-1515294898968-a408405d7674'
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))
img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

plt.figure(figsize=(8, 8))
plt.imshow(img[:,:,::-1])
plt.axis('off')
plt.show()

In [ ]:
height, width, _ = img.shape

frame_landmarks = get_frame_landmarks(img[:,:,::-1])
for landmark in frame_landmarks:
    x = int(landmark[0] * width)
    y = int(landmark[1] * height)
    cv2.circle(img, (x, y), 10, (0, 255, 0), -1)

plt.figure(figsize=(8, 8))
plt.imshow(img[:,:,::-1])
plt.axis('off')
plt.show()

<h3> Test on video

In [ ]:
with open('/kaggle/input/mutemotion-output/WLASL_parsed_data.json', 'r') as json_file:
    data = json.load(json_file)
    
test = data[40]
video_landmarks = get_video_landmarks(test['video_path'],test['frame_start'],test['frame_end'])

output_path = '/kaggle/working/landmarks_test.mp4'
draw_landmarks(test['video_path'], output_path, video_landmarks, test['frame_start'],test['frame_end'])

In [ ]:
os.remove('/kaggle/working/landmarks_test.mp4')

---

<div id="Encoding">
    
**<h2>Data Encoding   (Done Once)**

In [ ]:
npy_dir = '/kaggle/working/landmarks'
os.makedirs(npy_dir, exist_ok=True)

In [ ]:
try:
    for i in tqdm(range(len(data)), ncols=100):
        npy_path = os.path.join(npy_dir, f'{i}.npy')
        if os.path.exists(npy_path): continue
        video_path = data[i]['video_path']
        start = data[i]['frame_start']
        end = data[i]['frame_end']
        
        try:
            video_landmarks = get_video_landmarks(video_path, start, end)
            np.save(npy_path, video_landmarks)
            
        except Exception as e:
            print(f"\nError encoding {video_path}\n{e}")
            continue   
        clear_output(wait=True)

except KeyboardInterrupt:
    print("\nLoading process interrupted by user.")

In [ ]:
landmarks_dict = {}

for filename in os.listdir(npy_dir):
    if filename.endswith('.npy'):
        key = filename.split('.')[0]
        landmarks = np.load(os.path.join(npy_dir, filename), allow_pickle=True)
        landmarks_dict[key] = landmarks

np.savez_compressed('/kaggle/working/landmarks_V2.npz', **landmarks_dict)

FileLink(r'landmarks_V2.npz')

In [ ]:
# shutil.rmtree(npy_dir)
# os.remove('/kaggle/working/landmarks_V2.npz')

---

<div id="Loading">
    
**<h2>Data Loading**

In [ ]:
landmarks_dict = np.load('/kaggle/input/mutemotion-output/landmarks_V2.npz', allow_pickle=True)

In [ ]:
with open('/kaggle/input/mutemotion-output/WLASL_parsed_data.json', 'r') as json_file:
    data = json.load(json_file)

In [ ]:
def load_data(split, labels=None, max_labels=None, max_samples=None):
    
    if labels is not None:
        X = [landmarks_dict[k] for k in landmarks_dict.keys() if data[int(k)]['split'] == split and data[int(k)]['gloss'] in labels]
        Y = [data[int(k)]['gloss'] for k in landmarks_dict.keys() if data[int(k)]['split'] == split and data[int(k)]['gloss'] in labels]
    
    elif max_samples is not None:
        X = [landmarks_dict[k] for k in landmarks_dict.keys() if data[int(k)]['split'] == split][:max_samples]
        Y = [data[int(k)]['gloss'] for k in landmarks_dict.keys() if data[int(k)]['split'] == split][:max_samples]
    
    elif max_labels is not None:
        label_counts = {}
        for k in landmarks_dict.keys():
            label = data[int(k)]['gloss']
            label_counts[label] = label_counts.get(label, 0) + 1
        
        top_labels = sorted(label_counts, key=label_counts.get, reverse=True)[:max_labels]
        X = [landmarks_dict[k] for k in landmarks_dict.keys() if data[int(k)]['gloss'] in top_labels and data[int(k)]['split'] == split]
        Y = [data[int(k)]['gloss'] for k in landmarks_dict.keys() if data[int(k)]['gloss'] in top_labels and data[int(k)]['split'] == split]
        
    else:
        X = [landmarks_dict[k] for k in landmarks_dict.keys() if data[int(k)]['split'] == split]
        Y = [data[int(k)]['gloss'] for k in landmarks_dict.keys() if data[int(k)]['split'] == split]
    
    return X, Y

In [ ]:
X_train, Y_train = load_data('train', max_labels=250)
X_val, Y_val = load_data('val', max_labels=250)
X_test, Y_test = load_data('test', max_labels=250)

In [ ]:
len(X_train), len(X_val), len(X_test)

In [ ]:
len(np.unique(Y_train)), len(np.unique(Y_val)), len(np.unique(Y_test))

In [ ]:
np.all(np.unique(Y_val) in np.unique(Y_train)), np.all(np.unique(Y_test) in np.unique(Y_train))

In [ ]:
# free space
del landmarks_dict, data
gc.collect()

---

<div id="Augmentation">
    
**<h2> Data Augmentation**

<h3> Rotation Augmentations

In [ ]:
def rotate(data, rotation_matrix):
    frames, landmarks, _ = data.shape
    center = np.array([0.5, 0.5, 0])
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data = data.reshape(-1, 3)
    data[non_zero] -= center
    data[non_zero] = np.dot(data[non_zero], rotation_matrix.T)
    data[non_zero] += center
    data = data.reshape(frames, landmarks, 3)
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data

def rotate_z(data):
    angle = np.random.choice([np.random.uniform(-30, -10),
                              np.random.uniform(10, 30)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])
    return rotate(data, rotation_matrix)

def rotate_y(data):
    angle = np.random.choice([np.random.uniform(-30, -10),
                              np.random.uniform(10, 30)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    return rotate(data, rotation_matrix)

def rotate_x(data):
    angle = np.random.choice([np.random.uniform(-30, -10),
                              np.random.uniform(10, 30)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [1, 0, 0],
        [0, np.cos(theta), -np.sin(theta)],
        [0, np.sin(theta), np.cos(theta)]
    ])
    return rotate(data, rotation_matrix)

<h3> Other Augmentations

In [ ]:
def zoom(data):
    factor = np.random.uniform(0.8, 1.2)
    center = np.array([0.5, 0.5])
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data[non_zero[:, 0], non_zero[:, 1], :2] = (
        (data[non_zero[:, 0], non_zero[:, 1], :2] - center) * factor + center
    )
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data

def shift(data):
    x_shift = np.random.uniform(-0.2, 0.2)
    y_shift = np.random.uniform(-0.2, 0.2)
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data[non_zero[:, 0], non_zero[:, 1], 0] += x_shift
    data[non_zero[:, 0], non_zero[:, 1], 1] += y_shift
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data

def mask(data):
    frames, landmarks, _ = data.shape
    num_hands = int(0.3 * 42)
    num_rest = int(0.6 * (landmarks - 42))

    mask = np.zeros(landmarks, dtype=bool)
    indices = np.concatenate([
        np.random.choice(42, num_hands, replace=False),
        np.random.choice(landmarks - 42, num_rest, replace=False) + 42
    ])
    mask[indices] = True
    data[:, mask] = 0
    return data

def hflip(data):
    data[:, :, 0] = 1 - data[:, :, 0]
    return data

def speedup(data):
    return data[::2]

<h3> Apply Augmentations

In [ ]:
def apply_augmentations(data):
    aug_functions = [rotate_x, rotate_y, rotate_z, zoom, shift, mask, hflip, speedup]
    np.random.shuffle(aug_functions)
    counter = 0
    for fun in aug_functions:
        if np.random.rand() < 0.5:
            data = fun(data)
            counter += 1
    
    if counter == 0:
        data = apply_augmentations(data)

    return data

In [ ]:
def augment(X, Y, num=None):
    X_aug = X.copy()
    Y_aug = Y.copy()
    
    if num == None:
        for i in tqdm(range(len(Y)), ncols=100):
            num_aug = np.random.choice([1, 2, 3])
            for n in range(num_aug):
                X_aug.append(apply_augmentations(X[i].copy()))
                Y_aug.append(Y[i])
    elif num > 0:
        for i in tqdm(range(len(Y)), ncols=100):
            for n in range(num):
                X_aug.append(apply_augmentations(X[i].copy()))
                Y_aug.append(Y[i])

    return X_aug, Y_aug

In [ ]:
X_train_aug, Y_train_aug = augment(X_train, Y_train)

In [ ]:
len(X_train_aug), len(X_train_aug[0]), len(X_train_aug[0][0]), len(X_train_aug[0][0][0])

In [ ]:
# free space
del X_train, Y_train
gc.collect()

<h3> Test Augmentation (Not necessary when training)

In [ ]:
frame_landmarks = np.expand_dims(frame_landmarks, axis=0)

In [ ]:
frame_landmarks.shape

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

for landmark in frame_landmarks[0]:
    x = int(landmark[0] * img.shape[1])
    y = int(landmark[1] * img.shape[0])
    cv2.circle(img, (x, y), 10, (0, 255, 0), -1)
    
axs[0].set_title('Original Image')
axs[0].imshow(img[:, :, ::-1])
axs[0].axis('off')

augmented_landmarks = apply_augmentations(frame_landmarks.copy())
img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

for landmark in augmented_landmarks[0]:
    x = int(landmark[0] * img.shape[1])
    y = int(landmark[1] * img.shape[0])
    cv2.circle(img, (x, y), 10, (0, 255, 0), -1)

axs[1].set_title('Augmented Image')
axs[1].imshow(img[:, :, ::-1])
axs[1].axis('off')

plt.tight_layout()
plt.show()

<h3> Data Permutation

In [ ]:
permutation_train = list(range(len(Y_train_aug)))
np.random.shuffle(permutation_train)
X_train_aug = [X_train_aug[i] for i in permutation_train]
Y_train_aug = [Y_train_aug[i] for i in permutation_train]

permutation_val = list(range(len(Y_val)))
np.random.shuffle(permutation_val)
X_val = [X_val[i] for i in permutation_val]
Y_val = [Y_val[i] for i in permutation_val]

permutation_test = list(range(len(Y_test)))
np.random.shuffle(permutation_test)
X_test = [X_test[i] for i in permutation_test]
Y_test = [Y_test[i] for i in permutation_test]

---

<div id="Preprocessing">
    
**<h2>Data Preprocessing**

<h3>
    
**Method 1:** Sequencing

In [ ]:
def sequences(X, Y, length=30, step=1, pad=0):
    X_sequences = []
    Y_sequences = []

    for inputs, label in zip(X, Y):
        num = inputs.shape[0]

        if num < length:
            padding = length - num
            inputs = np.pad(
            inputs, ((0, padding), (0, 0), (0, 0)),
            mode='constant', constant_values=pad
            )
            num = length

        for start in range(0, num - length + 1, step):
            end = start + length
            sequence = inputs[start:end]
            X_sequences.append(sequence)
            Y_sequences.append(label)

    X_sequences = np.array(X_sequences)
#     Y_sequences = np.array(Y_sequences)
    return X_sequences, Y_sequences

In [ ]:
X_val_seq, Y_val_seq = sequences(X_val, Y_val, length=60, step=20, pad=-100)
X_test_seq, Y_test_seq = sequences(X_test, Y_test, length=60, step=20, pad=-100)
X_train_seq, Y_train_seq = sequences(X_train_aug, Y_train_aug, length=60, step=20, pad=-100)

In [ ]:
X_train_seq.shape, X_test_seq.shape, X_val_seq.shape

In [ ]:
nlp = spacy.load('en_core_web_sm')

Y_val_seq = np.array([nlp(label).vector for label in Y_val_seq])
Y_test_seq = np.array([nlp(label).vector for label in Y_test_seq])
Y_train_seq = np.array([nlp(label).vector for label in Y_train_seq])

In [ ]:
Y_train_seq.shape

In [ ]:
# label_encoder = LabelEncoder()

# Y_val_seq = label_encoder.fit_transform(Y_val_seq)
# Y_test_seq = label_encoder.fit_transform(Y_test_seq)
# Y_train_seq = label_encoder.fit_transform(Y_train_seq)

In [ ]:
# free space
del X_train_aug, Y_train_aug, X_val, Y_val, X_test, Y_test
gc.collect()

<h3>
    
**Method 2:** Padding

In [ ]:
def padding(X, Y, length=None, pad=0):
    if length is None:
        length = max(len(x) for x in X)
    
    X_padded = []
    for x in X:
        if len(x) > length:
            X_padded.append(x[:length]) #truncate
        else:
            pad_length = length - len(x)
            X_padded.append(np.pad(
                x, ((0, pad_length), (0, 0), (0, 0)),
                mode='constant', constant_values=pad
            ))
            
    X_padded = np.array(X_padded)
    Y = np.array(Y)
    return X_padded, Y

In [ ]:
X_val_pad, Y_val_pad = padding(X_val, Y_val, length=200, pad=-100)
X_test_pad, Y_test_pad = padding(X_test, Y_test, length=200, pad=-100)
X_train_pad, Y_train_pad = padding(X_train_aug, Y_train_aug, length=200, pad=-100)

In [ ]:
X_train_pad.shape, X_test_pad.shape, X_val_pad.shape

In [ ]:
label_encoder = LabelEncoder()

Y_val_pad = label_encoder.fit_transform(Y_val_pad)
Y_test_pad = label_encoder.fit_transform(Y_test_pad)
Y_train_pad = label_encoder.fit_transform(Y_train_pad)

In [ ]:
# free space
del X_train_aug, Y_train_aug, X_val, Y_val, X_test, Y_test
gc.collect()

---

<div id="Models">
    
**<h2> Models**

<h2>Hady's Model

<h2>Kamel's Model

In [ ]:
gc.collect()
tf.keras.backend.clear_session()
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
physical_devices

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=-100, input_shape=(60, 180, 3)),
    tf.keras.layers.Reshape((60, 180 * 3)),

    tf.keras.layers.Conv1D(64, 1, strides=1, padding='valid', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.DepthwiseConv1D(3, strides=1, padding='valid', depth_multiplier=1, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv1D(64, 1, strides=1, padding='valid', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.DepthwiseConv1D(5, strides=2, padding='valid', depth_multiplier=4, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.MaxPool1D(2, 2),

    tf.keras.layers.Conv1D(256, 1, strides=1, padding='valid', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.DepthwiseConv1D(3, strides=1, padding='valid', depth_multiplier=1, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv1D(256, 1, strides=1, padding='valid', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.DepthwiseConv1D(5, strides=2, padding='valid', depth_multiplier=4, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.GlobalAvgPool1D(),
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Dense(96)
])

In [ ]:
checkpoint_filepath = '/kaggle/working/Kamel_Checkpoints/'
os.makedirs(checkpoint_filepath, exist_ok=True)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_filepath, 'model_{epoch:02d}.h5'),
    save_weights_only=True,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
)

lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=50,
    restore_best_weights=True
)

model.compile(
    loss='mse',
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler),
    metrics=['accuracy']
)

In [ ]:
hist = model.fit(
    X_train_seq, Y_train_seq,
    validation_data=(X_val_seq, Y_val_seq),
    epochs=1000,
    batch_size=128,
    callbacks=[model_checkpoint, early_stopping]
)

In [ ]:
# epoch = 90
# model.load_weights(f'/kaggle/working/Kamel_Checkpoints/model_{epoch}.h5')
test_loss, test_accuracy = model.evaluate(X_test_seq, Y_test_seq)

In [ ]:
import shutil
shutil.rmtree('/kaggle/working/Kamel_Checkpoints')

In [ ]:
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']

# Plotting loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_loss, 'g', label='Training loss')
plt.plot(val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting accuracy
plt.subplot(1, 2, 2)
plt.plot(train_acc, 'g', label='Training accuracy')
plt.plot(val_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
img = plt.imread('model_plot.png')
plt.figure(figsize=(40, 30))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
name = '12-5'
model_filepath = '/kaggle/working/Kamel_Models'
os.makedirs(model_filepath, exist_ok=True)

model.save(model_filepath + f'/{name}')
model.save(model_filepath + f'/{name}.h5')

converter = tf.lite.TFLiteConverter.from_saved_model(model_filepath + f'/{name}')
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS,
  tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()
with open(model_filepath + f'/{name}.tflite', 'wb') as f:
    f.write(tflite_model)

<h2>Samir's Model

<h2>Nour's Model